In [3]:
import os
import numpy as np
import cv2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
import tensorflow as tf

In [4]:
# Parameters
IMG_SIZE = (128, 128)  # Size to which images will be resized before being fed into the model.
BATCH_SIZE = 8
EPOCHS = 50
KNOWN_DRIVERS_PATH = 'Known Drivers'

In [5]:
# Function to preprocess image array directly (for live camera feed)
def preprocess_image_array(img_array, target_size):
    img = cv2.resize(img_array, target_size)
    img = img.astype('float32') / 255.0  # Normalize the image pixel values to the range [0, 1].
    return img

In [8]:
def load_dataset(dataset_path, img_size):
    images = []
    labels = []
    driver_names = []

    for img_name in os.listdir(dataset_path):
        if img_name.lower().endswith(('.jpg', '.jpeg')):  # Ensure we're loading only image files
            img_path = os.path.join(dataset_path, img_name)
            
            # Load the image using OpenCV
            img_array = cv2.imread(img_path)
            
            if img_array is not None:
                img_array = preprocess_image_array(img_array, img_size)
                images.append(img_array)
                
                # Extract the driver's name from the image name (without file extension)
                driver_name = os.path.splitext(img_name)[0]
                labels.append(driver_name)
                
                if driver_name not in driver_names:
                    driver_names.append(driver_name)

    images = np.array(images)
    labels = np.array(labels)
    return images, labels, driver_names

In [9]:
# Load dataset
images, labels, driver_names = load_dataset(KNOWN_DRIVERS_PATH, IMG_SIZE)


# images: Will store all the processed image data in a NumPy array.
# labels: Will store the corresponding labels (driver names) in a NumPy array.
# driver_names: Will store the unique driver names as a list

In [ ]:
# Check if the labels and images are loaded correctly
if len(labels) == 0 or len(images) == 0:
    print("No images or labels were loaded. Please check the 'Known Drivers' folder.")
else:
    # Encode labels
    label_encoder = LabelEncoder()  #Encodes string labels to numeric values.
    labels_encoded = label_encoder.fit_transform(labels) # Fits the encoder and transforms labels into numeric form.
    labels_categorical = to_categorical(labels_encoded) #Converts numeric labels to one-hot encoded format

In [ ]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels_categorical, test_size=0.2, random_state=42)

In [ ]:
# Build the CNN model
#Convolutional layer with specified number of filters, kernel size, activation function, and padding.
def build_model(input_shape, num_classes):  
    inputs = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x) # reduce spatial dimensions.
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    return model

In [ ]:
# Compile the model
input_shape = IMG_SIZE + (3,) #input_shape: Specifies the shape of the input images (height, width, channels).
num_classes = len(driver_names)
model = build_model(input_shape, num_classes)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=BATCH_SIZE, epochs=EPOCHS)

Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 173ms/step - accuracy: 0.0000e+00 - loss: 4.3753 - val_accuracy: 0.0000e+00 - val_loss: 3.8607
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 138ms/step - accuracy: 0.0627 - loss: 3.8383 - val_accuracy: 0.0000e+00 - val_loss: 3.9260
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 138ms/step - accuracy: 0.0598 - loss: 3.8211 - val_accuracy: 0.0000e+00 - val_loss: 4.1106
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 145ms/step - accuracy: 0.1331 - loss: 3.6932 - val_accuracy: 0.0000e+00 - val_loss: 4.5010
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 138ms/step - accuracy: 0.1025 - loss: 3.6213 - val_accuracy: 0.0000e+00 - val_loss: 4.7707
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 152ms/step - accuracy: 0.0817 - loss: 3.4450 - val_accuracy: 0.0000e+00 - val_loss: 5.2574
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 138ms/step - accuracy: 0.1491 - loss: 3.1328 - val_accuracy: 0.0000e+00 - val_loss: 4.8902
Epoch 8/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 139ms/step - accuracy: 0.3094 - loss: 2.8381 - 

In [ ]:
# Save the model
model.save('driver_verification_model.h5')

In [ ]:

# # Load the model from file
# model = load_model('driver_verification_model.h5')

In [ ]:
# Function to recognize driver from an image array
def recognize_driver(image_array, model, label_encoder):
    img_array = preprocess_image_array(image_array, IMG_SIZE)
    img_array = np.expand_dims(img_array, axis=0)
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    confidence = predictions[0][predicted_class_index]

    if confidence > 0.5:  # Adjust threshold as needed
        driver_name = label_encoder.inverse_transform([predicted_class_index])[0] #Converts the predicted class index back to a driver name.
        return driver_name, confidence
    else:
        return None, None

In [ ]:
# Inference using live camera feed
def live_camera_inference(model, label_encoder):
    cap = cv2.VideoCapture(0)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        driver_name, confidence = recognize_driver(frame, model, label_encoder)
        
        if driver_name:
            text = f"Authorized Driver: ({confidence:.2f})"
        else:
            text = "He is not authorized to drive"
        
        cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0) if driver_name else (0, 0, 255), 2)
        cv2.imshow('Driver Verification', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
# Perform inference
live_camera_inference(model, label_encoder)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━